# Run Bash Agent with HuggingFace Model

This notebook runs the trained CLI agent using local HuggingFace model inference.

## Prerequisites

- Run `02_grpo_training.ipynb` first to train and save the model
- Model checkpoint should be at `outputs/grpo_langgraph_cli/merged_model`

## Features

- **Structured tool calling**: Uses JSON-based tool calls (not legacy code block parsing)
- **Human-in-the-loop**: All commands require user confirmation before execution
- **Security**: Command allowlist and injection protection
- **Conversation history**: Full context maintained across turns

## Step 1: Setup and Configuration

In [1]:
import os
import sys
import json

# Add bash_agent to path for imports
sys.path.insert(0, os.path.join(os.getcwd(), "bash_agent"))

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.7.1+cu126
CUDA available: True
GPU: NVIDIA H100 80GB HBM3


In [2]:
# Import configuration from bash_agent
from config import Config

# Create config with notebook-specific settings
config = Config()

# Override model path if needed (uncomment to change)
# config.model_path = "outputs/grpo_langgraph_cli/merged_model"

print(f"Model path: {config.model_path}")
print(f"Root directory: {config.root_dir}")
print(f"Allowed commands: {config.allowed_commands}")

Model path: /home/ubuntu/Build_a_Computer_Use_Agent_with_Synthetic_Data/outputs/grpo_langgraph_cli/merged_model
Root directory: /home/ubuntu/Build_a_Computer_Use_Agent_with_Synthetic_Data
Allowed commands: ['cd', 'cp', 'ls', 'cat', 'find', 'touch', 'echo', 'grep', 'pwd', 'mkdir', 'wget', 'sort', 'head', 'tail', 'du', 'wc', 'file', 'langgraph']


## Step 2: Import the Bash Tool

Import from `bash_agent/bash.py` - provides secure command execution with:
- Command allowlist
- Injection protection
- Working directory tracking

In [3]:
# Import Bash tool from bash_agent
from bash import Bash

# Initialize the bash tool with security features
bash = Bash(config)
print(f"Bash tool initialized. Working directory: {bash.cwd}")

Bash tool initialized. Working directory: /home/ubuntu/Build_a_Computer_Use_Agent_with_Synthetic_Data


## Step 3: Import Message Handler and LLM Interface

Import from `bash_agent/helpers.py`

In [4]:
# Import Messages and HuggingFaceLLM from bash_agent
from helpers import Messages, HuggingFaceLLM

print("Imported Messages and HuggingFaceLLM from bash_agent.helpers")

Imported Messages and HuggingFaceLLM from bash_agent.helpers


In [5]:
# HuggingFaceLLM was imported above from bash_agent.helpers
# It provides:
#   - Local model inference with HuggingFace transformers
#   - JSON tool call parsing from model responses
#   - Conversion of structured commands to bash commands
print("HuggingFaceLLM ready (imported from bash_agent.helpers)")

HuggingFaceLLM ready (imported from bash_agent.helpers)


## Step 4: Load the Trained Model

In [ ]:
# Verify model exists
model_path = config.model_path
if not os.path.exists(model_path):
    print(f"ERROR: Model not found at {model_path}")
    print("Please run 02_grpo_training.ipynb first to train and save the model.")
else:
    print(f"Model found at: {model_path}")
    print(f"\nContents:")
    for f in os.listdir(model_path)[:10]:
        print(f"  - {f}")

In [7]:
# Load the model
llm = HuggingFaceLLM(config)

Loading model from: /home/ubuntu/Build_a_Computer_Use_Agent_with_Synthetic_Data/outputs/grpo_langgraph_cli/merged_model


`torch_dtype` is deprecated! Use `dtype` instead!
Skipping import of cpp extensions due to incompatible torch version 2.7.1+cu126 for torchao version 0.15.0             Please see https://github.com/pytorch/ao/issues/2919 for more info


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded on cuda


## Step 5: Test the Model

Quick test to verify the model generates proper JSON tool calls.

In [10]:
# Test queries - use the JSON system prompt (matches training!)
test_queries = [
    "Create a new project using the react-agent template",
    "Start the dev server on port 8080 without opening a browser",
    "Build a Docker image and tag it as myapp:v2",
]

# IMPORTANT: Use json_system_prompt - this is what the model was trained with!
print("Using json_system_prompt (matches training):")
print(config.json_system_prompt[:200] + "...")
print("=" * 60)

for query in test_queries:
    print(f"\n[Query] {query}")
    
    # Create fresh messages with the JSON system prompt (not generic bash prompt)
    test_messages = Messages(config.json_system_prompt)
    test_messages.add_user_message(query)
    
    response, tool_calls = llm.query(test_messages)
    
    # Clean up response for display
    display_response = response
    if "</think>" in display_response:
        display_response = display_response.split("</think>")[-1].strip()
    
    if tool_calls:
        for tc in tool_calls:
            args = json.loads(tc["function"]["arguments"])
            print(f"[Tool Call] {tc['function']['name']}: {args.get('cmd', args)}")
    else:
        print("[Tool Call] None detected")
    
    print("-" * 40)

Using json_system_prompt (matches training):
You are an expert CLI assistant for the LangGraph Platform CLI.

Translate user requests into structured JSON tool calls.

Available commands:
- new: Create project (flags: template, path)
- dev: Star...

[Query] Create a new project using the react-agent template
[Tool Call] exec_bash_command: langgraph new --template react-agent
----------------------------------------

[Query] Start the dev server on port 8080 without opening a browser
[Tool Call] exec_bash_command: langgraph dev --port 8080 --no-browser
----------------------------------------

[Query] Build a Docker image and tag it as myapp:v2
[Tool Call] exec_bash_command: langgraph build -t myapp:v2
----------------------------------------


## Step 6: Interactive Agent Loop

Run the full agent with human-in-the-loop confirmation.

**Commands:**
- Type your request and press Enter
- When a command is proposed, type `y` to execute or `n` to decline
- Type `quit` or `exit` to stop

In [11]:
def confirm_execution(cmd: str) -> bool:
    """Ask user to confirm command execution."""
    response = input(f"    Execute '{cmd}'? [y/N]: ").strip().lower()
    return response == "y" or response == "yes"


def run_agent_loop():
    """Main agent interaction loop."""
    
    # Initialize conversation with JSON system prompt (matches training)
    messages = Messages(config.json_system_prompt)
    
    print("\n" + "=" * 60)
    print("Bash Computer Use Agent (HuggingFace)")
    print("=" * 60)
    print(f"Model: {config.model_path}")
    print(f"Working directory: {bash.cwd}")
    print("Type 'quit' or 'exit' to stop.")
    print("Type 'clear' to reset conversation.")
    print("=" * 60 + "\n")
    
    while True:
        try:
            user_input = input(f"['{bash.cwd}'] > ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\n\nShutting down. Bye!")
            break
        
        if user_input.lower() in ["quit", "exit"]:
            print("\nShutting down. Bye!")
            break
        
        if user_input.lower() == "clear":
            messages.clear()
            print("Conversation cleared.\n")
            continue
        
        if not user_input:
            continue
        
        # Add context about current directory
        user_with_context = f"{user_input}\nCurrent working directory: `{bash.cwd}`"
        messages.add_user_message(user_with_context)
        
        # Agent loop - may involve multiple tool calls
        while True:
            print("\nThinking...")
            
            try:
                response, tool_calls = llm.query(messages)
            except Exception as e:
                print(f"Error querying model: {e}")
                break
            
            # Clean response for display
            display_response = response.strip()
            if "</think>" in display_response:
                display_response = display_response.split("</think>")[-1].strip()
            
            if display_response:
                messages.add_assistant_message(display_response)
            
            # Process tool calls
            if tool_calls:
                for tc in tool_calls:
                    function_name = tc["function"]["name"]
                    function_args = json.loads(tc["function"]["arguments"])
                    tool_id = tc["id"]
                    
                    if function_name != "exec_bash_command" or "cmd" not in function_args:
                        tool_result = {"error": "Incorrect tool or function argument"}
                    else:
                        command = function_args["cmd"]
                        print(f"\nProposed command: {command}")
                        
                        if confirm_execution(command):
                            tool_result = bash.exec_bash_command(command)
                            
                            if tool_result.get("stdout"):
                                print(f"\nOutput:\n{tool_result['stdout']}")
                            if tool_result.get("stderr"):
                                print(f"\nError:\n{tool_result['stderr']}")
                            if tool_result.get("error"):
                                print(f"\nError:\n{tool_result['error']}")
                        else:
                            tool_result = {"error": "The user declined to execute this command."}
                    
                    messages.add_tool_message(json.dumps(tool_result), tool_id)
            else:
                # No tool calls - show assistant message and break
                if display_response:
                    print(f"\n{display_response}")
                print("-" * 60)
                break

print("Agent loop defined. Run the next cell to start the interactive agent.")

Agent loop defined. Run the next cell to start the interactive agent.


In [12]:
# Start the interactive agent
# Note: This cell requires interactive input. Stop with 'quit' or 'exit'
run_agent_loop()


Bash Computer Use Agent (HuggingFace)
Model: /home/ubuntu/Build_a_Computer_Use_Agent_with_Synthetic_Data/outputs/grpo_langgraph_cli/merged_model
Working directory: /home/ubuntu/Build_a_Computer_Use_Agent_with_Synthetic_Data
Type 'quit' or 'exit' to stop.
Type 'clear' to reset conversation.


Thinking...

Proposed command: langgraph new --template react-agent /home/ubuntu/Build_a_Computer_Use_Agent_with_Synthetic_Data

Thinking...

Okay, the user asked to start a new LangGraph project, and I called the 'new' command with the template 'react-agent' and the provided path. But the tool response says the user declined to execute the command. Hmm, maybe the user didn't actually want to proceed with creating the project. I should check if there was a misunderstanding or if the user changed their mind.

Wait, the user's original request was "Can you start a new langgraph project?" and I assumed they wanted to proceed. But the tool's response indicates a decline. Maybe the user didn't confirm 

## Summary

This notebook imports from the `bash_agent` module:

| Import | Source | Description |
|--------|--------|-------------|
| `Config` | `bash_agent/config.py` | Model path, security settings, system prompt |
| `Bash` | `bash_agent/bash.py` | Secure command execution with allowlist |
| `Messages` | `bash_agent/helpers.py` | Conversation history management |
| `HuggingFaceLLM` | `bash_agent/helpers.py` | Local model inference with JSON parsing |

### bash_agent Module Structure

```
bash_agent/
├── config.py      # Configuration class
├── bash.py        # Bash tool with security features
├── helpers.py     # Messages and HuggingFaceLLM classes
├── prompts.py     # System prompts (JSON-based, no legacy)
└── main_hf.py     # CLI entry point for the agent
```

### Key Features

- **Structured tool calling**: JSON-based tool calls (not code block parsing)
- **Human-in-the-loop**: All commands require user confirmation
- **Security**: Command allowlist and injection protection
- **Conversation history**: Full context maintained across turns

### Next Steps

- Train longer for better accuracy (increase `max_steps` in training)
- Add more commands to the allowlist in `bash_agent/config.py`
- Run from CLI: `python bash_agent/main_hf.py`
- Deploy as a server with vLLM for production use